In [1]:
from tqdm import tqdm
import codecs
import math
import numpy as np
import os
import pandas as pd
import re

print(os.getcwd())

C:\Users\User\Desktop\Ricardo\KnowledgeGraph\scripts


# Some Reference

* Regular Expression Tester: https://regex101.com/
* Some Re Tutorials: https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285

# Processing Steps
* Divide paragraphs into sentences by Chinese punctuations (“，”, “。”, “、”, “？”, “?”).
* Segment sentences into tokens with relative dependencies and POSs (Part of Speech).
* Remove tokens with certain type of dependency and POS (i.e. PUNCT, x, PARENTHESISCATEGORY)
* Remove unnecessary tokens
  * punctuation
  * word between two bracket
* Extract possible tokens as candidates of entities and relations by its dependency and POS category,
  * with regular expression (Entity, POS, dependencies): “^[V]”, “^[N]”,
  * with certain dependencies and POS type,
  * with given entity dictionaries (entity, POS, dependencies),
  * with certain conjuction chars (["的", "、", "之", "及", "與"]) ,
* Clean up candidate list again by removing unnecessary tokens (conjunction characters). 
  * 晶圓的 -> 晶圓, 的製程 -> 製程, issues caused by adjectives.
* Remove stopwords
* Remove entity and relation with only one and more than 8 chars
* Caculate TFiDF/Frequency and sort

<br>

#### Optional
* Conclude frequency of occurrence of each node/edge candidate, keeping the first 20 ranks only.

# Configurations
### Extract possible tokens as candidates of entities and relations by its dependency and POS category

In [2]:
# paths
token_path = "../results/210408_result/210408_dataset_monpaResult.csv"
entity_saving_path = "../results/210408_result/210408_dataset_entity_result_MONPA.csv"
relation_saving_path = "../results/210408_result/210408_dataset_relation_result_MONPA.csv"
main_result_saving_path = "../results/210408_result/210408_dataset_main_result_MONPA.csv"
filtered_result_path = "../results/210408_result/210408_dataset_filtered_result_MONPA.csv"

# token_path = "../results/210330_result/210330_dataset_spaCyResult.csv"
# entity_saving_path = "../results/210330_result/210330_dataset_entity_result_spaCy.csv"
# relation_saving_path = "../results/210330_result/210330_dataset_relation_result_spaCy.csv"
# main_result_saving_path = "../results/210330_result/210330_dataset_main_result_spaCy.csv"
# filtered_result_path = "../results/210408_result/210408_dataset_filtered_result_spaCy.csv"

In [3]:
# MONPA
relation_dependencies_possible_list = []
relation_pos_possible_list = ["VH", "VC", "VJ", "VA"]
relation_pos_re = "^[V]"
entity_dependencies_possible_list = []
entity_pos_possible_list = ["Na", "Nv", "Neu", "Nes", "Nf", "Ng", "Nh", "Neqa", "Nep", "Ncd", "FW", "DE"]
entity_pos_re = "^[N]"
splitter_pos_list = ["COMMACATEGORY", "PERIODCATEGORY"]

# spaCy
# relation_dependencies_possible_list = ["ROOT", "nmod:prep", "prep", "agent", ]
# relation_pos_possible_list = ["VERB"]
# relation_pos_re = "^[V]"
# entity_dependencies_possible_list = ["compound:nn", "nsubj", "dep", "dobj"]
# entity_pos_possible_list = ["NOUN", "PROPN"]
# entity_pos_re = "^[N]"
# punct_pos_list = ["PUNCT"]

# Common, usually entity
sentences_splitter = ["，", "。", "！", "!", "？", "?", "；", ";", "."] + ["\r\n" * i for i in range(0, 100)]
bracket_entity_list_first = ["(", "（", "[", "［", "{", "｛", "<", "＜", "〔", "【", "〖", "《", "〈"]
bracket_entity_list_last = [")", "）", "]", "］", "}", "｝", ">", "＞", "〕", "】", "〗", "》", "〉"]
punct_entity_list = [" " * i for i in range(0, 100)]
conjuction_entity_list = ["的", "、", "之", "及", "與"]
not_entity_relation_list = ["的", "、", "之", "及", "與", "\r\n \r\n ", "\r\n \r\n  "] +\
[" " * i for i in range(0, 100)] + ["\n" * i for i in range(0, 100)]

# Read in Tokens (Preparation beforehand)

In [4]:
# read in tokens

dataToken = pd.read_csv(token_path, encoding="utf8")

# append a column for data if dependencies column does not exist (monpa case)
if len(dataToken.columns) < 3:
    dataToken.loc[:, "Dependecies"] = np.nan

print(len(dataToken))
print(dataToken.head(5))

143131
  Segmented Element  POS  Dependecies
0                12  Neu          NaN
1                 吋   Nf          NaN
2                晶圓   Na          NaN
3                後段  Ncd          NaN
4                製程   Na          NaN


# Divide by Chinese Seperators
# Segment sentences into tokens with relative dependencies and POSs (Part of Speech).
# Remove tokens with certain type of dependency and POS (i.e. PUNCT, x, PARENTHESISCATEGORY)
# Remove unnecessary tokens

In [5]:
# devide token by seperators
total_entity_list = []
sentence_entity_list = []
total_label_list = []
sentence_label_list = []
total_dependencies_list = []
sentence_dependencies_list = []


# devide tokens by chinese punctuations
for index, entityElement in enumerate(dataToken.iloc[:, 0]):
    if entityElement not in sentences_splitter:
        sentence_entity_list.append(entityElement)
        sentence_label_list.append(dataToken.iloc[index, 1])
        sentence_dependencies_list.append(dataToken.iloc[index, 2])
    else:
        total_entity_list.append(sentence_entity_list)
        total_label_list.append(sentence_label_list)
        total_dependencies_list.append(sentence_dependencies_list)
        sentence_entity_list = []
        sentence_label_list = []
        sentence_dependencies_list = []
    
print("list length (Sentences): ", len(total_entity_list), len(total_label_list), len(total_dependencies_list))


# Clean up tokens and eliminate unneccesary tokens
for sentenceIndex, sentences in enumerate(total_entity_list):
    for tokenIndex, tokens in enumerate(sentences):
        # delete elements that's between two bracket
        if sentences[tokenIndex] in bracket_entity_list_first:
            left_bracket_index = bracket_entity_list_first.index(sentences[tokenIndex])
            findingLimit =  (tokenIndex+11) if (tokenIndex+11) <= (len(sentences)) else len(sentences)
            for findingLeftIndex in range(tokenIndex+1, findingLimit):
                if sentences[findingLeftIndex] == bracket_entity_list_last[left_bracket_index]:
                    for removalIndex in range(tokenIndex, findingLeftIndex+1):
#                         print(total_entity_list[sentenceIndex][removalIndex])
                        total_entity_list[sentenceIndex][removalIndex] = ""
                        total_label_list[sentenceIndex][removalIndex]= ""
                        total_dependencies_list[sentenceIndex][removalIndex] = ""
                    break
        elif sentences[tokenIndex] in punct_entity_list or sentences[tokenIndex] in sentences_splitter:
            # set token that fit certain POS type to ""
            total_entity_list[sentenceIndex][tokenIndex] = ""
            
    total_entity_list[sentenceIndex] = list(filter(("").__ne__, total_entity_list[sentenceIndex]))
    total_label_list[sentenceIndex]  = list(filter(("").__ne__, total_label_list[sentenceIndex]))
    total_dependencies_list[sentenceIndex] = list(filter(("").__ne__, total_dependencies_list[sentenceIndex]))
                
print(total_entity_list[0] , total_label_list[0], total_dependencies_list[0])
print("list length: ", len(total_entity_list), len(total_label_list), len(total_dependencies_list))

list length (Sentences):  9887 9887 9887
['12', '吋', '晶圓', '後段', '製程', '之', '發展', '趨勢', '探討', '前瞻', '封裝', '系列', '2002年', '08月', '05日', '星期一', '瀏覽', '人次', '：', '台灣', '半導體', '產業', '結構', '完整'] ['Neu', 'Nf', 'Na', 'Ncd', 'Na', 'DE', 'Nv', 'Na', 'VE', 'FW', 'FW', 'FW', 'Nv', 'Na', 'Na', 'FW', 'FW', 'FW', 'FW', 'FW', 'Nd', 'Nd', 'Nd', 'FW', 'Nd', 'Na', 'FW', 'VC', 'Na', 'FW', 'FW', 'FW', 'FW', 'FW', 'FW', 'FW', 'FW', 'LOC', 'Na', 'Na', 'Na', 'VH'] [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
list length:  9887 9887 9887


# Extract possible tokens as candidates of entities and relations by its dependency and POS category.
#  Clean up candidate list again by removing unnecessary tokens (conjunction characters). 

In [6]:
# get entity pairs and relations

# get relations
all_relations_list = []
all_relations_index = []
all_element_flatten = []
all_relation_flatten = []

for sentenceIndex, sentences in enumerate(total_entity_list):
    relation_list = []
    relation_index_list = []
    for tokenIndex, token in enumerate(sentences):
        if len(sentences) > 1: 
            if total_dependencies_list[sentenceIndex][tokenIndex] in relation_dependencies_possible_list or\
            total_label_list[sentenceIndex][tokenIndex] in relation_pos_possible_list or\
            re.match(relation_pos_re, str(total_label_list[sentenceIndex][tokenIndex]), flags=re.IGNORECASE) != None:
                relation_list.append(token)
                relation_index_list.append(tokenIndex)
    all_relations_list.append(relation_list)
    all_relations_index.append(relation_index_list)


# get entities
all_reformatted_entities = []
all_reformatted_index_list = []
for sentenceIndex, sentences in enumerate(total_entity_list):
    entity_index_list = []
    possible_entities = []
    reformatted_entities = []
    reformatted_index_list = []
    for tokenIndex, token in enumerate(sentences):
        if len(sentences) > 1:
            if total_dependencies_list[sentenceIndex][tokenIndex] in entity_dependencies_possible_list or\
            total_label_list[sentenceIndex][tokenIndex] in entity_pos_possible_list or\
            re.match(entity_pos_re, str(total_label_list[sentenceIndex][tokenIndex]), flags=re.IGNORECASE) != None or\
            token in conjuction_entity_list:
                entity_index_list.append(tokenIndex)
                possible_entities.append(token)
    # combine token if situate next to each other
#     print(entity_index_list, possible_entities)
    if len(possible_entities) > 0:
        combine_entity_name = possible_entities[0]
        for possibleIndex, possibleElement in enumerate(entity_index_list):
#             print(possible_entities)
            
            isContinuous = False
            if possibleIndex != 0:
                if possibleElement == entity_index_list[possibleIndex - 1] + 1:
                    isContinuous = True
                    combine_entity_name += str(possible_entities[possibleIndex])
                else:
                    isContinuous = False
            
                if isContinuous == False:
                    reformatted_entities.append(combine_entity_name)
                    reformatted_index_list.append(entity_index_list[possibleIndex-1])
                    combine_entity_name = possible_entities[possibleIndex]
                    
            if possibleIndex == (len(entity_index_list) - 1):
                reformatted_entities.append(combine_entity_name)
                reformatted_index_list.append(possibleElement)
                
#             print(combine_entity_name)     
#     print(reformatted_entities)
    all_reformatted_entities.append(reformatted_entities)
    all_reformatted_index_list.append(reformatted_index_list)
    
# flatten section
# remove unneccessary conjuction words
for index, element in enumerate(all_reformatted_entities):
    
    for elementIndex, elementSingle in enumerate(element):
        # remove single conjuction char in first or last position
        if elementSingle[0] in conjuction_entity_list:
            element[elementIndex] = elementSingle[1:]
        elif elementSingle[-1] in conjuction_entity_list:
            element[elementIndex] = elementSingle[:-1]
            
        all_element_flatten.append(element[elementIndex])
        
    for relationIndex, relationSingle in enumerate(all_relations_list[index]):
        # remove single conjuction char in first or last position
        if relationSingle[0] in conjuction_entity_list:
            all_relations_list[index][relationIndex] = relationSingle[1:]
        elif relationSingle[-1] in conjuction_entity_list:
            all_relations_list[index][relationIndex] = relationSingle[:-1]
            
        all_relation_flatten.append(all_relations_list[index][relationIndex])
                      
        
# print(all_relations_list, len(all_relations_list), "\n")
# print(all_reformatted_entities, len(all_reformatted_entities), )
print(len(all_relations_list), len(all_relations_index), len(all_reformatted_entities), len(all_reformatted_index_list))

# print out parsing result
for index, element in enumerate(all_reformatted_entities[:15]):
    print(element, all_reformatted_index_list[index], all_relations_list[index], all_relations_index[index])
    
data_result = pd.DataFrame({
    "Entity":all_reformatted_entities,
    "EntityIndex":all_reformatted_index_list,
    "Relation":all_relations_list,
    "RelationIndex":all_relations_index
})

9887 9887 9887 9887
['12吋晶圓後段製程之發展趨勢', '前瞻封裝系列2002年08月05日星期一瀏覽人次：台灣半導體產業結構完整'] [7, 23] ['探討'] [8]
['其', '晶片製造合作模式'] [1, 8] ['緊密', '延伸'] [2, 3]
['數年間', '半導體製程供應鏈'] [3, 11] ['過去', '建立', '完整'] [1, 5, 7]
['附加價值', '製程發展'] [6, 9] ['持續', '朝向', '高'] [1, 2, 3]
['成本效益與縮短產品', '時程的考量'] [6, 10] ['上市'] [7]
['12吋晶圓廠', '晶圓代工下一波的競爭主力'] [2, 12] ['成為'] [4]
['國際整合元件大廠', '產能', '趨勢下'] [4, 7, 12] ['委外', '代工'] [8, 9]
['晶圓', '', '表現'] [3, 5, 7] ['造就', '代工', '亮麗'] [0, 4, 6]
['半導體產業的競爭優勢'] [7] ['建立'] [1]
['國內晶圓', '兩', '龍頭', '積體電路', '科學園區', '科學園區', '12吋晶圓代工製造廠'] [1, 3, 5, 9, 15, 18, 27] ['代工', '大', '開始', '投資', '興建'] [2, 4, 20, 21, 22]
['小部份產能'] [3] ['進行', '試產'] [1, 4]
['國際級的整合元件大廠', '12吋晶圓的發展趨勢'] [5, 14] ['前進'] [15]
['後段的封裝製造廠', '整合性', '服務'] [4, 7, 10] ['使得', '提供', '化'] [0, 6, 9]
['製程技術'] [4] ['強化'] [2]
['晶圓尺寸增加後'] [5] ['滿足'] [1]


In [7]:
# conculde dataframe with all tokens
        
for notIndex, notElement in enumerate(not_entity_relation_list):
    all_element_flatten = list(filter((notElement).__ne__, all_element_flatten))
    all_relation_flatten = list(filter((notElement).__ne__, all_relation_flatten))
        
data_entity = pd.DataFrame({
    "Element":all_element_flatten,
})

data_relation = pd.DataFrame({
    "Element":all_relation_flatten,
})

# Remove stopwords
# Remove entity and relation with only one char and too many chars.

In [8]:
# import stopwords list

stopword_list = []

for fileIndex, fileElement in enumerate(os.listdir("./stopwords/")):
    if fileElement[-2:] != "md":
        data_temp = pd.read_csv("./stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")
        stopword_list += data_temp.iloc[:, 0].tolist()
        
stopword_list = list(dict.fromkeys(stopword_list))
        
print("停用詞數目：", len(stopword_list), "\n\n", stopword_list[-50:])     

C:\ProgramData\Anaconda3\envs\monpa\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


停用詞數目： 2003299 

 ["t's", 'twice', 'unfortunately', 'unless', 'unlikely', 'unto', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'value', 'various', 'viz', 'vs', 'want', 'wants', "wasn't", 'way', "we'd", 'welcome', "we'll", 'went', "we're", "weren't", "we've", "what's", "where's", "who's", 'willing', 'wish', 'wonder', "won't", "wouldn't", 'yes', "you'd", "you'll", "you're", "you've", 'zero', 'zt', 'ZT', 'zz', 'ZZ', '準備', '覆雜', '心裏', '註意', '裏面']


In [9]:
# remove stop words and entity & relation that only occur once

data_list = [data_entity, data_relation]

for dataIndex, dataElement in enumerate(data_list):
    drop_index = []
    for rowIndex, rowElement in enumerate(tqdm(dataElement.iloc[:, 0])):
        if rowElement in stopword_list or len(str(rowElement)) <= 1 or len(str(rowElement)) >= 8:
            drop_index.append(rowIndex)
    # drop entity/relations that are inside stopword list
    print("Original Length:", len(dataElement))
    dataElement.drop(index=drop_index, inplace=True)
    print("After Removal:", len(dataElement))
    

  0%|          | 6/17909 [00:00<05:57, 50.13it/s]

Original Length: 23302
After Removal: 16093


100%|██████████| 17909/17909 [06:04<00:00, 49.07it/s]

Original Length: 17909
After Removal: 10229


# Caculate TFiDF/Frequency and sort

In [10]:
# conculde a list with frequency

def ratiolize(x):
    x = str(np.around(x * 100, decimals=2)) + "%"
    return x

data_entity_value_count = data_entity.value_counts(ascending=False).to_frame()
data_entity_value_count.reset_index(inplace=True)
data_entity_value_count = data_entity_value_count.rename(columns = {'index':'Entity'})
data_entity_value_count = data_entity_value_count.rename(columns = {0:'Count'})
data_entity_value_count.loc[:, "Ratio"] = np.around(data_entity_value_count.iloc[:, 1] / len(data_entity_value_count),
                                                    decimals=4)
# data_entity_value_count.loc[:, "Ratio"] = data_entity_value_count.loc[:, "Ratio"].apply(ratiolize)

data_relation_value_count = data_relation.value_counts(ascending=False).to_frame()
data_relation_value_count.reset_index(inplace=True)
data_relation_value_count = data_relation_value_count.rename(columns = {'index':'Relation'})
data_relation_value_count = data_relation_value_count.rename(columns = {0:'Count'})
data_relation_value_count.loc[:, "Ratio"] = np.around(data_relation_value_count.iloc[:, 1] / len(data_relation_value_count),
                                                      decimals=4)
# data_relation_value_count.loc[:, "Ratio"] = data_relation_value_count.loc[:, "Ratio"].apply(ratiolize)


'''
calculate tfidf
'''
documentNum = len(os.listdir("../data/"))
data_all = [data_entity_value_count, data_relation_value_count]

# gather all document
textDocuments = []
for fileIndex, fileElement in enumerate(os.listdir("../data/")):
    textElement = ""
    file = codecs.open("../data/" + fileElement, 'r', encoding='utf8', errors='ignore')
    for textIndex, textLines in enumerate(file):
        textElement += textLines
    textDocuments.append(textElement)


for dataIndex, dataElement in enumerate(data_all):
    for rowIndex, rowElement in enumerate(tqdm(dataElement.iloc[:, 0])):
        document_count = 0
        for textIndex, textElement in enumerate(textDocuments):
            # check if entity/relation exists in the doucument
            if rowElement in textElement:
                document_count += 1
        # assign document frequency
        dataElement.loc[rowIndex, "DocumentFrequency"] = document_count
        # assign tfidf
        dataElement.loc[rowIndex, "tfiDF"] = np.around(dataElement.loc[rowIndex, "Count"] * \
            math.log(documentNum / (dataElement.loc[rowIndex, "DocumentFrequency"] + 1), 10), decimals=4)
        
# data_entity_value_count.sort_values(by=["tfiDF"], ascending=False, inplace=True)
# data_relation_value_count.sort_values(by=["tfiDF"], ascending=False, inplace=True)

print(len(data_entity), data_entity_value_count[:10], "\n")
print(len(data_relation), data_relation_value_count[:10])

100%|██████████| 3220/3220 [00:01<00:00, 1982.39it/s]

16093   Element  Count   Ratio  DocumentFrequency    tfiDF
0      晶圓    135  0.0127               26.0   0.0000
1      技術     90  0.0085               26.0   0.0000
2      製程     83  0.0078               27.0  -1.3109
3      晶片     68  0.0064               22.0   4.7352
4      產品     66  0.0062               22.0   4.5960
5      成本     48  0.0045               17.0   8.4524
6      表面     45  0.0042               20.0   4.9115
7      測品     41  0.0039                3.0  34.0015
8      過程     40  0.0038               20.0   4.3658
9      設備     40  0.0038               22.0   2.7854 

10229   Element  Count   Ratio  DocumentFrequency    tfiDF
0      回收    132  0.0410                5.0  86.2241
1      製造    127  0.0394               25.0   2.0816
2      利用    113  0.0351                6.0  66.2480
3      進行    107  0.0332                7.0  56.5253
4      包括    102  0.0317               17.0  17.9613
5      成長     92  0.0286               16.0  18.4842
6      提供     88  0.0273        

In [11]:
# Saving result without filtering
data_entity_value_count.to_csv(entity_saving_path, encoding="utf8", index=None, quoting=False)
data_relation_value_count.to_csv(relation_saving_path, encoding="utf8", index=None, quoting=False)
data_result.to_csv(main_result_saving_path, encoding="utf8", index=None, quoting=False)

# Filter out most frequent edge and relation

In [12]:
data_entity_value_count = pd.read_csv(entity_saving_path, encoding="utf8")
data_relation_value_count = pd.read_csv(relation_saving_path, encoding="utf8")
data_result = pd.read_csv(main_result_saving_path, encoding="utf8")

# take only first 20 places
data_entity_filter = data_entity_value_count.loc[:, "Element"].tolist()[:20]
data_relation_filter = data_relation_value_count.loc[:, "Element"].tolist()[:20]

abandonIndexList = []

for rowIndex, rowItem in enumerate(data_result.iloc[:, 0]):
    need_to_abandon = True
    entityList = rowItem.replace("'", "").replace(" ", "").replace("[", "").replace("]", "").split(",")[:-1]
    relationList = data_result.iloc[rowIndex, 2].replace("'", "").replace(" ", "").replace("[", "").replace("]", "").split(",")[:-1]
    
    for entityIndex, entityElement in enumerate(entityList):
        if entityElement in data_entity_filter:
            need_to_abandon = False
            break
    
    for relationIndex, relationElement in enumerate(relationList):
        if need_to_abandon == False:
            break
        if relationElement in data_relation_filter:
            need_to_abandon = False
            break
            
    if need_to_abandon == True:
        abandonIndexList.append(rowIndex)
            
data_result.drop(index=abandonIndexList, inplace=True)
print(data_result)

# export data
data_result.to_csv(filtered_result_path, encoding="utf8", index=None)

                                        Entity         EntityIndex  \
7                             ['晶圓', '', '表現']           [3, 5, 7]   
12                   ['後段的封裝製造廠', '整合性', '服務']          [4, 7, 10]   
40                              ['產能', '產品良率']              [3, 8]   
41                ['晶圓', '應用趨勢就晶圓尺寸的', '趨勢而言']          [0, 8, 11]   
51    ['12吋晶圓的後段製程', '吋晶圓的競爭優勢', '應用朝向', '趨勢']     [5, 14, 18, 22]   
...                                        ...                 ...   
9871                         [' 壞去', '達', '%']           [4, 7, 9]   
9873  ['凝器置於', '控制設備(', '設備', '碳吸 附床及吸收', '後']  [3, 9, 14, 20, 23]   
9874                ['冷凝處理', '廢氣成分中', '溫度之不同']          [1, 6, 10]   
9875        ['冷凝作用', '兩種方式在定溫', '增加系統之壓力2在定壓']          [1, 8, 16]   
9878                   ['冷凝器型式', '表面式及接觸式冷凝器']             [4, 11]   

                        Relation      RelationIndex  
7             ['造就', '代工', '亮麗']          [0, 4, 6]  
12             ['使得', '提供', '化']          [0, 6, 9]